# Data Mining (CpSc 8650) Course Project  Quality Evaluation of Skull Stripped Brain MRI Images

In [1]:
import os
import zipfile
import numpy as np
import tensorflow as tf
import nibabel as nib
import random
import csv
from scipy import ndimage
import pandas as pd
from tensorflow import keras
import tensorflow.keras.optimizers as optimizers
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tensorflow.keras import layers
import pickle
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
%matplotlib inline

In [2]:
# if tf.config.list_physical_devices('GPU'):
#     physical_devices = tf.config.list_physical_devices('GPU')
#     tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)
#     tf.config.experimental.set_virtual_device_configuration(physical_devices[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4000)])

In [3]:
# !conda create -n my_environment -y
# !source activate my_environment
# !pip install sklearn
# !pip install nibabel
#!wget --no-check-certificate https://dyslexia.computing.clemson.edu/BET_BSE/BET_BSE_DATA.zip

In [4]:
# load and return data 
def load_data(csv_file_path, test_size = 0.3, x_names = [], y_names = []):

    
    """
    Load training data and split it into training and validation set
    """
    print ("Loading data from: ",csv_file_path)
    #reads CSV file into a single dataframe variable
    data_df = pd.read_csv(csv_file_path, names=x_names+y_names, skiprows=1)

    #yay dataframes, we can select rows and columns by their names
    X = data_df[x_names].values
    #and our steering commands as our output data
    y = data_df[y_names ].values

    return X, y

def read_nifti_file(filepath):
    """Read and load volume"""
    # Read file
    scan = nib.load(filepath)
    # Get raw data
    scan = scan.get_fdata()
    return scan


def normalize(volume):
    """Normalize the volume"""
    min = -1000
    max = 400
    volume[volume < min] = min
    volume[volume > max] = max
    volume = (volume - min) / (max - min)
    volume = volume.astype("float32")
    return volume


def resize_volume(img):
    """Resize across z-axis"""
    # Set the desired depth
    desired_depth = 64
    desired_width = 128
    desired_height = 128
    # Get current depth
    current_depth = img.shape[-1]
    current_width = img.shape[0]
    current_height = img.shape[1]
    # Compute depth factor
    depth = current_depth / desired_depth
    width = current_width / desired_width
    height = current_height / desired_height
    depth_factor = 1 / depth
    width_factor = 1 / width
    height_factor = 1 / height
    # Rotate
    img = ndimage.rotate(img, 90, reshape=False)
    # Resize across z-axis
    img = ndimage.zoom(img, (width_factor, height_factor, depth_factor), order=1)
    return img

def process_scan(path):
    global iteration
    """Read and resize volume"""
    # Read scan
    volume = read_nifti_file(path)
    # Normalize
    volume = normalize(volume)
    #Resize width, height and depth
    volume = resize_volume(volume)
    iteration+=1
    print(iteration)
    return volume

In [5]:
@tf.function
def rotate(volume):
    """Rotate the volume by a few degrees"""

    def scipy_rotate(volume):
        # define some rotation angles
        angles = [0,60,-60,-90,90,180,-180]
        # pick angles at random
        angle = random.choice(angles)
        # rotate volume
        volume = ndimage.rotate(volume, angle, reshape=False)
        volume[volume < 0] = 0
        volume[volume > 1] = 1
        return volume

    augmented_volume = tf.numpy_function(scipy_rotate, [volume], tf.float32)
    return augmented_volume


def train_preprocessing(volume, labels):
    """Process training data by rotating and adding a channel."""
    # Rotate volume
    volume = rotate(volume)
    volume = tf.expand_dims(volume, axis=3)
    return volume, labels


def validation_preprocessing(volume, label):
    """Process validation data by only adding a channel."""
    volume = tf.expand_dims(volume, axis=3)
    return volume, label



In [6]:
label_file_path = os.path.join(os.path.abspath(os.pardir), "dataset", "Label_file.csv")
X, y = load_data(label_file_path, x_names = ["Filename"], y_names = ["Recognizable-Facial-Feature","Brain-Feature-Loss"])

Loading data from:  /home/rkaranj/Data Mining/New/dataset/Label_file.csv


In [7]:
y[y=='Yes'] = 1
y[y=='No'] = 0

In [8]:
def get_model(width=128, height=128, depth=64):
        """Build a 3D convolutional neural network model."""

        inputs = keras.Input((width, height, depth, 1))

        x = layers.Conv3D(filters=16, kernel_size=3, activation="relu")(inputs)
        x = layers.MaxPool3D(pool_size=2)(x)
        x = layers.BatchNormalization()(x)
        x = layers.Dropout(0.3)(x)

        x = layers.Conv3D(filters=32, kernel_size=3, activation="relu")(x)
        x = layers.MaxPool3D(pool_size=2)(x)
        x = layers.BatchNormalization()(x)
        x = layers.Dropout(0.5)(x)

        x = layers.Conv3D(filters=64, kernel_size=3, activation="relu")(x)
        x = layers.MaxPool3D(pool_size=2)(x)
        x = layers.BatchNormalization()(x)
        x = layers.Dropout(0.5)(x)

        x = layers.GlobalAveragePooling3D()(x)
        x = layers.Dense(units=256, activation="relu")(x)
        x = layers.Dropout(0.3)(x)

        outputs = layers.Dense(units=2, activation="sigmoid")(x)

        # Define the model.
        model = keras.Model(inputs, outputs, name="3dcnn")
        return model

In [ ]:
from sklearn.model_selection import KFold
skf = KFold(n_splits=5,shuffle=True,random_state=1)
skf.get_n_splits(X, y)
i = 0

model = get_model(width=128, height=128, depth=64)
for train_index, test_index in skf.split(X):
    i+=1
    print("Fold:"+str(i))
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    iteration=0
    #print(len(y_train), len(y_test))
    # X_train_data = np.array([process_scan(os.path.join(os.path.abspath(os.pardir), "dataset", "files", path[0]+".gz")) for path in X_train])
    # X_valid_data = np.array([process_scan(os.path.join(os.path.abspath(os.pardir), "dataset", "files", path[0]+".gz")) for path in X_test])
    
    outfileTrain = "Fold " + str(i) + " Checkpoints.pkl"
    outfileTest = "Fold " + str(i) + " Checkpoints test set.pkl"
    #X_train_valid = [X_valid_data]
#     with open(outfile, "wb") as f:
#         pickle.dump(X_train_valid, f)
    #print(len(X_valid_data), len(y_test))
    # Load the objects back
    with open(outfileTrain, "rb") as f:
        [X_train_data, abc] = pickle.load(f)
        
    with open(outfileTest, "rb") as f:
        [X_valid_data] = pickle.load(f)
    print("Data Loaded")
    # Define data loaders.
    train_loader = tf.data.Dataset.from_tensor_slices((X_train_data,y_train.astype("float32")))
    validation_loader = tf.data.Dataset.from_tensor_slices((X_valid_data, y_test.astype("float32")))
    
    batch_size = 24
    # Augment the on the fly during training.
    train_dataset = (
        train_loader.shuffle(len(X_train_data))
        .map(train_preprocessing)
        .batch(batch_size)
        .prefetch(2)
    )
    
    # Only rescale.
    validation_dataset = (
        validation_loader.shuffle(len(X_valid_data))
        .map(validation_preprocessing)
        .batch(batch_size)
        .prefetch(2)
    )
    # Build model.
    print(train_dataset)
    # Compile model.
    initial_learning_rate = 0.00001
    lr_schedule = optimizers.schedules.ExponentialDecay(
        initial_learning_rate, decay_steps=100000, decay_rate=0.96, staircase=True
    )
    model.compile(
        loss="binary_crossentropy",
        optimizer=optimizers.Adam(learning_rate=lr_schedule),
        metrics=["acc"],
    )

    
    # Define callbacks.
    checkpoint_cb = keras.callbacks.ModelCheckpoint(
        "3d_image_classification_relu.h5", save_best_only=True
    )

    early_stopping_cb = keras.callbacks.EarlyStopping(monitor="val_acc", patience=15)

    # Train the model, doing validation at the end of each epoch
    epochs = 100
    model.fit(
        train_dataset,
        validation_data=validation_dataset,
        epochs=epochs,
        callbacks=[checkpoint_cb, early_stopping_cb],
    )
    
    fig, ax = plt.subplots(1, 2, figsize=(20, 3))
    ax = ax.ravel()

    for i, metric in enumerate(["acc", "loss"]):
        ax[i].plot(model.history.history[metric])
        ax[i].plot(model.history.history["val_" + metric])
        ax[i].set_title("Model {}".format(metric))
        ax[i].set_xlabel("epochs")
        ax[i].set_ylabel(metric)
        ax[i].legend(["train", "val"])

Fold:1
Data Loaded
<PrefetchDataset element_spec=(TensorSpec(shape=<unknown>, dtype=tf.float32, name=None), TensorSpec(shape=(None, 2), dtype=tf.float32, name=None))>
Epoch 1/100
44/44 [==============================] - 131s 3s/step - loss: 0.6059 - acc: 0.7824 - val_loss: 0.7390 - val_acc: 0.4221
Epoch 2/100
44/44 [==============================] - 129s 3s/step - loss: 0.5348 - acc: 0.9046 - val_loss: 0.8060 - val_acc: 0.4221
Epoch 3/100
44/44 [==============================] - 129s 3s/step - loss: 0.4706 - acc: 0.9361 - val_loss: 0.8861 - val_acc: 0.4221
Epoch 4/100
44/44 [==============================] - 129s 3s/step - loss: 0.4241 - acc: 0.9504 - val_loss: 0.9744 - val_acc: 0.4221
Epoch 5/100
44/44 [==============================] - 129s 3s/step - loss: 0.3853 - acc: 0.9656 - val_loss: 1.0603 - val_acc: 0.4221
Epoch 6/100
44/44 [==============================] - 129s 3s/step - loss: 0.3457 - acc: 0.9695 - val_loss: 1.1476 - val_acc: 0.4221
Epoch 7/100
44/44 [======================

In [4]:
from tensorflow import keras

model = keras.models.load_model('3d_image_classification_relu.h5')

## Displaying Image

In [3]:

image = X_train_data[0]
print("Dimension of the CT scan is:", image.shape)
plt.imshow(np.squeeze(image[:, :, 30]), cmap="gray")
plt.show()

NameError: name 'X_train_data' is not defined

In [ ]:
def plot_slices(num_rows, num_columns, width, height, data):
    """Plot a montage of 20 CT slices"""
    data = np.rot90(np.array(data))
    data = np.transpose(data)
    data = np.reshape(data, (num_rows, num_columns, width, height))
    rows_data, columns_data = data.shape[0], data.shape[1]
    heights = [slc[0].shape[0] for slc in data]
    widths = [slc.shape[1] for slc in data[0]]
    fig_width = 12.0
    fig_height = fig_width * sum(heights) / sum(widths)
    f, axarr = plt.subplots(
        rows_data,
        columns_data,
        figsize=(fig_width, fig_height),
        gridspec_kw={"height_ratios": heights},
    )
    for i in range(rows_data):
        for j in range(columns_data):
            axarr[i, j].imshow(data[i][j], cmap="gray")
            axarr[i, j].axis("off")
    plt.subplots_adjust(wspace=0, hspace=0, left=0, right=1, bottom=0, top=1)
    plt.show()


# Visualize montage of slices.
# 4 rows and 10 columns for 100 slices of the CT scan.
plot_slices(4, 10, 128, 128, image[:, :, :40])

### Predict with a image

In [ ]:
file = process_scan('../dataset/files/IXI002-Guys-0828-T1_bse_less_s5_r1.nii.gz')
file = tf.expand_dims(file, axis=0)

prediction = model.predict(file)


# yes,no = prediction[0] * 100, (1-prediction[0]) *100

# print("Yes=",yes[0],"No=",no[0])
# if(yes[0]>50):
#     print("Facial Feature Recognizable")
# else:
#     print("Facial Feature not recognizable")

In [ ]:
prediction[0][0]>0.5


In [ ]:
prediction[0][1]>0.5